In [1]:
import numpy as np
import pandas as pd
from utils.fraud_modeller import *
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Load data

In [2]:
from pickleObjects import *

In [3]:
path = './Data/'

X_train, X_test, y_train, y_test = loadObjects(path+'X_train'),loadObjects(path+'X_test'),loadObjects(path+'y_train'),loadObjects(path+'y_test')

Object loaded!
Object loaded!
Object loaded!
Object loaded!


# Scale Data

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

In [5]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

# Machine learning models

## 1. Deep Neural Net

In [6]:
from sklearn.metrics import roc_auc_score

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [8]:
X_train_scaled.shape[1]

306

In [63]:
model = Sequential()

model.add(Dense(units=150, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(units=74, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=36, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(units=2, activation='softmax'))

In [64]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [65]:
# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', 
                             monitor='val_loss', save_best_only=True)]

# Train neural network
history = model.fit(X_train_scaled, # Features
                      y_train, # Target vector
                      epochs=100, # Number of epochs
                      callbacks=callbacks, # Early stopping
                      verbose=1, # Print description after each epoch
                      batch_size=128, # Number of observations per batch
                      validation_data=(X_test_scaled, y_test)) # Data for evaluation

Train on 472432 samples, validate on 118108 samples
Epoch 1/100
472432/472432 [==============================] - 15s 31us/step - loss: 0.1284 - val_loss: 0.1167
Epoch 2/100
472432/472432 [==============================] - 15s 31us/step - loss: 0.1175 - val_loss: 0.1141
Epoch 3/100
472432/472432 [==============================] - 15s 32us/step - loss: 0.1131 - val_loss: 0.1098
Epoch 4/100
472432/472432 [==============================] - 15s 31us/step - loss: 0.1106 - val_loss: 0.1075
Epoch 5/100
472432/472432 [==============================] - 14s 30us/step - loss: 0.1082 - val_loss: 0.1063
Epoch 6/100
472432/472432 [==============================] - 15s 31us/step - loss: 0.1065 - val_loss: 0.1049
Epoch 7/100
472432/472432 [==============================] - 14s 30us/step - loss: 0.1050 - val_loss: 0.1027
Epoch 8/100
472432/472432 [==============================] - 15s 32us/step - loss: 0.1040 - val_loss: 0.1021
Epoch 9/100
472432/472432 [==============================] - 14s 30us/step -

In [66]:
classes = history.model.predict(X_test_scaled, batch_size=128)

In [67]:
roc_auc_score(y_test,classes)

0.8771390379340748